# Разработка модели контекстного перевода сокращений в текстах на русском языке
## 3. Сборка датасета

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

import pandas as pd
import numpy as np

import pickle
from collections import defaultdict

from tqdm import tqdm
import random

from sklearn.model_selection import train_test_split

from src.abbr import AbbrInfo, AbbrTree

SEED = 1

### Загрузка данных 

In [2]:
with open('../data/tokenized_abbr.pickle', 'rb') as f:
    tokenized_abbr = pickle.load(f)
tokenized_abbr

{('теория', 'механизм', 'и', 'машина'): 'тмм',
 ('бакинский', 'высокий', 'военный', 'училище'): 'бвву',
 ('служба', 'технический', 'эксплуатация', 'флот'): 'стэф',
 ('первичный', 'единица', 'отбор'): 'пео',
 ('группа', 'обеспечение', 'реализация', 'договор'): 'горд',
 ('московский', 'скутер', 'клуб'): 'мск',
 ('типоразмерный', 'ряд'): 'трр',
 ('самарский', 'губернский', 'дума'): 'сгд',
 ('наземный', 'пункт', 'наведение'): 'нпн',
 ('великий', 'теорема', 'ферма'): 'втф',
 ('можгинский', 'педагогический', 'колледж'): 'мпк',
 ('производственный', 'ремонтный', 'эксплуатационный', 'объединение'): 'прэо',
 ('ручной', 'регулировка', 'громкость'): 'ррг',
 ('беспилотный', 'крылатый', 'летательный', 'аппарат'): 'бкла',
 ('магнитоэлектрический', 'клапан', 'подача', 'топливо'): 'мкпт',
 ('производственный', 'издательский', 'комбинат'): 'пик',
 ('организационно', 'методический', 'документ'): 'омд',
 ('коммуникация', 'интернет'): 'коминтерн',
 ('уральский', 'государственный', 'горный', 'университет')

In [3]:
with open('../data/tokenized_texts.pickle', 'rb') as f:
    tokenized_texts = pickle.load(f)
tokenized_texts[0][:10]

['бой',
 'у',
 'сопоцкина',
 'и',
 'друскеник',
 'закончиться',
 'отступление',
 'германец',
 'неприятель',
 'приблизиться']

## Класс для работы с аббривиатурами

In [4]:
abbr_info = AbbrInfo(tokenized_abbr)

In [5]:
abbr_info.id2abbr[10], abbr_info.id2desc[10]

('мпк', ('можгинский', 'педагогический', 'колледж'))

In [9]:
abbr_info.abbr2desc_list["мф"]

[('международный', 'федерация'),
 ('медь', 'фосфористый'),
 ('мир', 'фантастика'),
 ('мой', 'футбол'),
 ('международный', 'факультет'),
 ('мебельный', 'фабрика'),
 ('микрофильм',),
 ('миллифарада',),
 ('местный', 'финансы'),
 ('московский', 'филиал'),
 ('микрофотометр',),
 ('монофосфат',),
 ('министерство', 'финансы', 'минфин'),
 ('маломерный', 'флот'),
 ('металлургический', 'факультет'),
 ('масляный', 'фильтр'),
 ('миллифот',),
 ('медианный', 'фильтрация'),
 ('международный', 'фонд')]

### Дерево аббривиатур

In [7]:
abbr_tree = AbbrTree(abbr_info)

In [8]:
abbr_tree.get_desc_tree()["министерство"]["строительство"]

{'предприятие': {'машиностроение': {'<id>': 651}},
 'топливный': {'предприятие': {'<id>': 5975}},
 '<id>': 16201,
 'московский': {'область': {'<id>': 23814}}}

## Замена на сокращения 

In [9]:
new_text, new_labels = abbr_tree.replace_by_abbr(tokenized_texts[7])

In [10]:
list(zip(new_text, new_labels))[:10]

[('япония', 0),
 ('принять', 0),
 ('решение', 0),
 ('разморозить', 0),
 ('кредит', 0),
 ('россия', 0),
 ('на', 0),
 ('сумма', 0),
 ('в', 0),
 ('полтора', 0)]

## Сохранение 

In [11]:
texts_replaced = []
labels_replaced = []
for text in tqdm(tokenized_texts):
    new_text, new_labels = abbr_tree.replace_by_abbr(text, p=0.5)
    texts_replaced.append(np.array(new_text))
    labels_replaced.append(np.array(new_labels))

100%|██████████| 796639/796639 [01:01<00:00, 13029.47it/s]


In [12]:
del tokenized_texts

### Разделение на тест и трейн 

In [13]:
train_texts, test_texts, train_labels, test_labels = train_test_split(texts_replaced, 
                                                                      labels_replaced,
                                                                      test_size=0.2, 
                                                                      random_state=SEED)

In [14]:
del texts_replaced, labels_replaced

### Сохранение данных 

In [15]:
with open('../data/train_texts.pickle', 'wb') as f:
    pickle.dump(train_texts, f)
with open('../data/test_texts.pickle', 'wb') as f:
    pickle.dump(test_texts, f)
    
with open('../data/train_labels.pickle', 'wb') as f:
    pickle.dump(train_labels, f)
with open('../data/test_labels.pickle', 'wb') as f:
    pickle.dump(test_labels, f)